## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [5]:
import pandas as pd
from pymystem3 import Mystem

df = pd.read_csv('data.csv')

print(df.info())
print()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Исходя из результата выполнения функции info() можно увидеть, что присутствуют пропуски в столбцах "days_employed" и "total_income". Также тип переменной, которая хранится в "days_employed", является числом с плавающей точкой, что является нелогичным для подсчета количества дней. Здесь же большинство значений отрицательны и встречаются положительные, которые являются явно ошибочными, поскольку слишком велики. При этом отрицательные числа в пересчете на года выглядят вполне себе нормальными, следовательно можно сделать вывод, что при выгрузке данных каким-то образом дни стали отрицательными.
В столбце "children" есть значения -1, которое, очевидно, ошибочно.
В столбце "education" встречаются дубликаты, записанные в разном регистре.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [30]:
#Поиск пропусов
mis_val_de        = df[df['days_employed'].isnull()]['children'].count()
mis_val_ti        = df[df['total_income'].isnull()]['children'].count()
mis_val_de_and_ti = df[(df['days_employed'].isnull()) & (df['total_income'].isnull())]['children'].count()

print()
print('Количество пропусков в столбце "days_employed" ' + str(mis_val_de))
print('Количество пропусков в столбце "total_income" ' + str(mis_val_ti))
print('Количество пропусков одновременно в столбце "days_employed" и в столбце "total_income" ' + str(mis_val_de_and_ti))
print()

#Замена пропусков в столбцах total_income и days_employed
total_income_median = df['total_income'].median()
df['total_income']  = df['total_income'].fillna(value=total_income_median)
valid_days          = df[df['days_employed'] < 0]
emp_days_mean       = valid_days['days_employed'].mean()
df['days_employed'] = df['days_employed'].fillna(value=emp_days_mean)

#Заменим также явно некорректные значения в days_employed
df.loc[df['days_employed'] > 0,'days_employed'] = emp_days_mean
#Уберем отрицательный знак
df['days_employed'] = (-1)*df['days_employed']

#Выделим только те строки, в которых количество детей неотрицательное
valid_children = df[df['children'] >= 0]
#Посчитаем медиану
children_median = valid_children['children'].median()
#Заменим отрицательные числа на медиану
df.loc[df['children'] < 0,'children'] = children_median

#Проверим, что пропусков больше нет
print(df.info())


Количество пропусков в столбце "days_employed" 2174
Количество пропусков в столбце "total_income" 2174
Количество пропусков одновременно в столбце "days_employed" и в столбце "total_income" 2174

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null float64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB
None


### Вывод

В результате выполнения кода в ячейке выше было выявлено, что пропуски в данной выборке находятся в столбцах days_employed и total_income, причем попарно и в количестве 2174. Прямое удаление всех строк с пропусками может привести к ошибочным выводам, потому необходимо их чем-то заполнить. На первый взгляд кажется, что данные столбцы не относятся к предмету анализа и следовательно пустые ячейки можно заполнить нулями. Однако, поскольку тип переменной в данных столбцах является количественным, то для заполнения пропусков можно воспользоваться характерным значением (средним или медианой). В столбце total_income мусора нет, потому напрямую вычисляем медиану и заполняем ею все пропуски (из-за классового неравенства доход сильно различается, следовательно надо использовать медиану). С days_employed таким подходом могут возникнуть проблемы, но при тщательном анализе выборки можно увидеть, что все положительные числа чрезмерно большие, а все отрицательные числа находятся в пределе от 0 до -15000, что в днях выглядит вполне правдоподобно. Значит надо выбрать все строки, где данная величина принимает отрицательные значения и вычислить среднее, которым и будут заполнены все соответствующие пропуски и некорректные значения. Также в этом столбце избавились от отрицательных знаков.
В столбце "children" все отрицательные значения заменили на медиану, расчитанную тольо по положительным значениям (медиану взяли по той причине, что в выборке есть 76 значений с 20 детьми, из-за этого среднее получилось бы завышеным).

### Замена типа данных

In [32]:
#Используется метод astype, поскольку to_numeric не позволяет преобразовать в целочисленный тип
df['days_employed'] = df['days_employed'].astype('int')
df['children']      = df['children'].astype('int')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

Производим преобразование типа из float в int для столбца days_employed, поскольку дни целесообразно исчислять целочисленными значениями, а для "children" это пришлось сделать из-за того, что тип поменялся при избвлении от отрицательных чисел.

### Обработка дубликатов

In [33]:
df['education'] = df['education'].str.lower()
print(df.duplicated().sum())
df = df.drop_duplicates()
df = df.dropna().reset_index(drop=True)
print(df.info())

71
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

Поиск дубликатов был произведен не по каким-то отдельным столбцам, а по всем сразу. Найдено всего 71. Если бы производили поиски по отдельным столбцам, то дубликатов было бы больше, поскольку многие столбцы хранят в себе категориальный тип переменных, которые естесственным образом будут повторяться от одной записи к другой. В связи с большим количеством записей дубликаты также некорректно искать по столбцам, хранящие количественный тип переменной. По ним дубликатов тоже будет гораздо больше. 
Перед самим поиском был применен метод lower() для столбца education. Без его применения количество дубликатов выявило бы меньше.
Дубликаты могут появляться из-за человеческого фактора.

### Лемматизация

In [9]:
m        = Mystem()
purposes = df['purpose'].unique()

#Сколько уникальных целей
num_of_purposes = len(purposes)
print(num_of_purposes)

#Лемматизируем каждое слово в каждой цели и вручную определяем какие имена категорий захватят все цели
purposes_lemms = []
for purpose in purposes:
        purposes_lemms.append(m.lemmatize(purpose))
    
categories = ['недвижимость','жилье','автомобиль','образование','свадьба']

38


### Вывод

Оказалось достаточно пяти категорий, чтобы распределить все цели. Хотя изначально уникальных целей было 38, каждая из которых состояла из нескольких слов, что значительно усложняло группировку данных по целям.

### Категоризация данных

In [36]:
#Функция, которая лемматизирует, передаваемую в качестве аргумента, цель и определяет то, к какой категории она принадлежит
def categ_def(purpose):
    purpose_lemmatized = m.lemmatize(purpose)
    for category in categories:
        if category in purpose_lemmatized:
            return category
    return 'не найдена'

#Тестирование функции
# print(categ_def('покупка жилья'))
# print(categ_def('приобретение автомобиля'))
# print(categ_def('дополнительное образование'))
# print(categ_def('на проведение свадьбы'))
# print(categ_def('покупка коммерческой недвижимости'))
#print(categ_def('полная чушь'))

#Добавление нового столбца в датафрейм 
df['category'] = df['purpose'].apply(categ_def)
#Проверка, что категоризация прошла успешно
print(df[df['category'] == 'не найдено'])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, category]
Index: []


### Вывод

Была написана функция, которая определяла к какой категории относится цель, записанная в столбце "purpose" и через apply был создан новый столбец "category".

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
#Вывод влияния количества детей на наличие долгов
print(df.groupby('children')['debt'].sum())
print()
#Вывод влияния наличия детей на общий доход
print(df.groupby('children')['total_income'].sum())

children
0     1064
1      444
2      194
3       27
4        4
5        0
20       8
Name: debt, dtype: int64

children
0     2.306405e+09
1     8.110769e+08
2     3.472471e+08
3     5.910097e+07
4     6.814214e+06
5     1.508460e+06
20    1.260944e+07
Name: total_income, dtype: float64


### Вывод

Очень явная здесь зависимость заключающаяся в том, что люди, у которых есть дети менее склонны к тому, чтобы залазить в долги(вероятной причиной здесь может быть материнский капитал). Даже несмотря на то, что заработок у них меньше и это понятно, ведь дети требуют внимания и потому меньше времени на то, чтобы зарабатывать деньги.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
#Вывод влияния семейного статуса на наличие долгов
print(df.groupby('family_status')['debt'].sum())
print()
#Вывод влияния семейного статуса на общий доход
print(df.groupby('family_status')['total_income'].sum())

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64

family_status
Не женат / не замужем    4.663834e+08
в разводе                2.002800e+08
вдовец / вдова           1.378901e+08
гражданский брак         6.834755e+08
женат / замужем          2.056733e+09
Name: total_income, dtype: float64


### Вывод

Люди, находящиеся в официальном браке явно имеют больше долгов, хотя и общий заработок у них больше. Это объясняется тем, что семейные пары в начале совместной жизни часто берут кредит на жилье. Однако, как показал предыдущий вывод, это можно исправить, если заводить детей:)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
def income_level(income):
    if income < 50000:
        return 'До 50тыс'
    elif 50000 < income < 100000:
        return 'От 50тыс до 100тыс'
    elif 100000 < income < 200000:
        return 'От 100тыс до 20тыс'
    elif 200000 < income < 500000:
        return 'От 200тыс до 500тыс'
    return 'Более 500тыс'

df['income_category'] = df['total_income'].apply(income_level)
result = df.groupby('income_category').agg({'debt':['sum','count']})
try:
    result['avg'] = result['debt']['sum'] / result['debt']['count']
except:
    print('Где-то появился 0 в знаменателе')
    
print(result)

                     debt              avg
                      sum  count          
income_category                           
Более 500тыс           14    222  0.063063
До 50тыс               23    372  0.061828
От 100тыс до 20тыс   1029  11925  0.086289
От 200тыс до 500тыс   344   4844  0.071016
От 50тыс до 100тыс    331   4091  0.080909


### Вывод

В среднем долгов больше у людей с с зарплатой "От 100тыс до 200тыс" и "От 50тыс до 100тыс".

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
result = df.groupby('category').agg({'debt':['sum','count']})
result['avg'] = result['debt']['sum'] / result['debt']['count']
print(result)

             debt             avg
              sum count          
category                         
автомобиль    403  4306  0.093590
жилье         308  4460  0.069058
недвижимость  474  6351  0.074634
образование   370  4013  0.092200
свадьба       186  2324  0.080034


### Вывод

Долги по жилью и недвижимости выплачивают исправнее по сравнению с автомобилями, образованием и свадьбами. Вероятно это связано с тем, что жилье и недвижимость это более серьезное вложение и потому люди относятся здесь к долгам ответственнее. В то время как автомобили покупают в кредит люди, которые не могут справляться с внезапными порывами, которые быстро остывают. Кредит на свадьбу это тоже то еще удовольствие, на которое решаются не особо ответственные и умные люди. Высокая задолженность по образованию может быть связана с тем, что оно на коммерческой основе достаточно дорогое, в то время как требует много сил и вложений без долгой окупаемости.

### Шаг 4. Общий вывод

Люди, которые находятся в официальном браке более склонны к тому, чтобы залазить в долги и это объяснимо, ведь семья требует больших вложений(жилье, автомобиль для работы или свадьба), которых вначале может и не быть. Однако, как показали цифры люди с детьми менее склонны к долгам. Вероятно это связано с материнским капиталом и с тем, что люди становятся более ответственными после появления ребенка, даже несмотря на то, что заработок начинает падать.
В долги менее склонны залезать люди либо с очень малым заработком, либо с очень большим.
Больше всего долгов по кредитам на автомобиль, а меньше всего по жилью.